In [1]:
!pip install transformers
from transformers import OpenAIGPTTokenizer, OpenAIGPTModel
import torch
import torch
from transformers import GPT2LMHeadModel
from transformers import GPT2Tokenizer
import pandas as pd
import nltk
nltk.download('stopwords')
import re
from bs4 import BeautifulSoup
import numpy as np
import torch
import transformers as ppb 
from transformers import OpenAIGPTTokenizer, OpenAIGPTModel
import torch

     |████████████████████████████████| 3.5 MB 5.4 MB/s 
     |████████████████████████████████| 67 kB 4.1 MB/s 
     |████████████████████████████████| 6.8 MB 33.6 MB/s 
     |████████████████████████████████| 596 kB 33.9 MB/s 
     |████████████████████████████████| 895 kB 24.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS =nltk.corpus.stopwords.words('english')

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

In [3]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [4]:
def gpt_fscore(text,model,tokenizer):
    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0) 
    outputs = model(input_ids)
    last_hidden_states = outputs[0]
    return last_hidden_states


In [5]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
path = "/content/drive/MyDrive/MWP(Words and Values in List).csv"
df = pd.read_csv(path)
df.head()

Mounted at /content/drive


,Unnamed: 0,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [6]:
df.iloc[:,1] = df.iloc[:,1].apply(clean_text)
print(df.iloc[:100,1])

0     constants alpha beta frac x alpha x+ beta frac...
1     length segment points 2a a4 4 1 2 sqrt 10 unit...
2     chewbacca 20 pieces cherry gum 30 pieces grape...
3     constants b let f x left begin array cl ax + b...
4             calculate sum 1 + 3 + 5 + cdots + 15 + 17
                            ...                        
95    largest value x satisfies equation sqrt 2x 4x ...
96    sum left dfrac 1 3 right + left dfrac 1 3 righ...
97    map 12centimeter length represents 72 kilomete...
98                      find sum values x 2 x23x2 4 x 4
99              given 0 f g 8 f x x2 + 8 g x x2 4 value
Name: Problem , Length: 100, dtype: object


In [7]:
train_tokens = list(map(lambda t:   tokenizer.tokenize(t)+['[CLS]'], df.iloc[:,1]))
train_tokens_ids = list(map(tokenizer.convert_tokens_to_ids,train_tokens))
b=np.zeros((np.shape(df)[0]))

In [8]:
for i in range(0,np.shape(df)[0]):
  b[i]=np.shape(train_tokens_ids[i])[0] 
pad=np.zeros((np.shape(df)[0],int(np.max(b))))    

In [9]:
for i in range(0,np.shape(df)[0]):
  a=train_tokens_ids[i]
  for j in range(0,np.shape(a)[0]):
    pad[i,j]=a[j]

In [10]:
pad.size

849328

In [11]:
pad1=pad[1:500,:] 
input_ids = torch.tensor(np.array(pad1))

In [12]:
with torch.no_grad():
  last_hidden_states = model(input_ids.long())

In [13]:
features = last_hidden_states[0][:,0,:].numpy()

In [14]:
features

array([[-30.196753, -29.171091, -32.159336, ..., -37.644638, -37.29271 ,
        -30.012392],
       [-31.484392, -30.829475, -33.50373 , ..., -39.24053 , -37.991467,
        -31.642157],
       [-27.759674, -26.771677, -29.062952, ..., -33.985764, -33.7796  ,
        -27.726143],
       ...,
       [-29.461073, -28.742573, -32.01738 , ..., -36.750614, -36.757538,
        -29.550879],
       [-29.461073, -28.742573, -32.01738 , ..., -36.750614, -36.757538,
        -29.550879],
       [-32.65207 , -31.735031, -34.869087, ..., -39.219807, -39.493015,
        -32.57699 ]], dtype=float32)

In [15]:
# convert array into dataframe
DF = pd.DataFrame(features)
  

In [16]:
from google.colab import files
DF.to_csv('gpt2_algebra.csv') 
files.download('gpt2_algebra.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>